In [ ]:
# Import necessary libraries
import dlt
from pyspark.sql.functions import col, when

# Configure Azure Storage credentials
adls_storage_name = "waterqualitydlsg"
adls_storage_access_key = "<storage_access_key>"

spark.conf.set(
  "fs.azure.account.key." + adls_storage_name + ".blob.core.windows.net", adls_storage_access_key
)

file_location = "wasbs://waterqualitydata@waterqualitydlsg.blob.core.windows.net"

# Bronze Layer
@dlt.table(name="bronze_table", comment="Raw water quality data in JSON format")
def bronze_table():
    bronze_df = spark.read.json(file_location)
    return bronze_df

# Silver Layer
@dlt.table(name="silver_table", comment="Cleaned water quality data with enhanced schema")
def silver_table():
    bronze_df = dlt.read("bronze_table")
    
    # Data Cleaning and Transformation
    silver_df = bronze_df.drop(
        "monitoringSiteIdentifierScheme", "observedPropertyDeterminandCode", "procedureAnalyticalMethod",
        "parameterSampleDepth", "resultObservationStatus", "remarks", "metadata_beginLifeSpanVersion",
        "metadata_statusCode", "metadata_observationStatus", "metadata_statements", "metadata_versionId",
        "resultStandardDeviationValue"
    ).withColumnRenamed("countryCode", "Country_Code") \
      .withColumnRenamed("monitoringSiteIdentifier", "Monitoring_SiteID") \
      .withColumnRenamed("parameterWaterBodyCategory", "Water_Body") \
      .withColumnRenamed("observedPropertyDeterminandLabel", "Determinand_Label") \
      .withColumnRenamed("procedureAnalysedMatrix", "Analyzed_Matrix") \
      .withColumnRenamed("resultUom", "Result_Unit") \
      .withColumnRenamed("phenomenonTimeReferenceYear", "Reference_Year") \
      .withColumnRenamed("parameterSamplingPeriod", "Sampling_Period") \
      .withColumnRenamed("procedureLOQValue", "LOQ_Value") \
      .withColumnRenamed("resultNumberOfSamples", "Num_of_Samples") \
      .withColumnRenamed("resultQualityNumberOfSamplesBelowLOQ", "Quality_Samples") \
      .withColumnRenamed("resultQualityMinimumBelowLOQ", "Quality_MinimumValue") \
      .withColumnRenamed("resultMinimumValue", "Minimum_Value") \
      .withColumnRenamed("resultQualityMeanBelowLOQ", "Quality_Meanvalue") \
      .withColumnRenamed("resultMeanValue", "Mean_Value") \
      .withColumnRenamed("resultQualityMaximumBelowLOQ", "Quality_MaximumValue") \
      .withColumnRenamed("resultMaximumValue", "Maximum_value") \
      .withColumnRenamed("resultQualityMedianBelowLOQ", "Quality_MedianValue") \
      .withColumnRenamed("resultMedianValue", "Median_Value")

    # Create the when-otherwise conditions for each country code
    country_name_expr = when(col("Country_Code") == "AT", "Austria") \
    .when(col("Country_Code") == "CZ", "Czech Republic") \
    .when(col("Country_Code") == "DE", "Germany") \
    .when(col("Country_Code") == "BE", "Belgium") \
    .when(col("Country_Code") == "ES", "Spain") \
    .when(col("Country_Code") == "SK", "Slovakia") \
    .when(col("Country_Code") == "SE", "Sweden") \
    .when(col("Country_Code") == "DK", "Denmark") \
    .when(col("Country_Code") == "IE", "Ireland") \
    .when(col("Country_Code") == "CH", "Switzerland") \
    .when(col("Country_Code") == "RO", "Romania") \
    .when(col("Country_Code") == "EL", "Greece") \
    .when(col("Country_Code") == "NO", "Norway") \
    .when(col("Country_Code") == "BG", "Bulgaria") \
    .when(col("Country_Code") == "SI", "Slovenia") \
    .when(col("Country_Code") == "FR", "France") \
    .when(col("Country_Code") == "IT", "Italy") \
    .when(col("Country_Code") == "PL", "Poland") \
    .when(col("Country_Code") == "LT", "Lithuania") \
    .when(col("Country_Code") == "HR", "Croatia") \
    .when(col("Country_Code") == "RS", "Serbia") \
    .when(col("Country_Code") == "LV", "Latvia") \
    .when(col("Country_Code") == "CY", "Cyprus") \
    .when(col("Country_Code") == "AL", "Albania") \
    .when(col("Country_Code") == "MK", "North Macedonia") \
    .when(col("Country_Code") == "BA", "Bosnia and Herzegovina") \
    .when(col("Country_Code") == "MT", "Malta") \
    .when(col("Country_Code") == "FI", "Finland") \
    .when(col("Country_Code") == "XK", "Kosovo") \
    .when(col("Country_Code") == "EE", "Estonia") \
    .when(col("Country_Code") == "TR", "Turkey") \
    .when(col("Country_Code") == "LU", "Luxembourg") \
    .when(col("Country_Code") == "HU", "Hungary") \
    .when(col("Country_Code") == "PT", "Portugal") \
    .when(col("Country_Code") == "NL", "Netherlands") \
    .when(col("Country_Code") == "IS", "Iceland") \
    .when(col("Country_Code") == "LI", "Liechtenstein") \
    .when(col("Country_Code") == "ME", "Montenegro") \
    .otherwise("Unknown")  # Default value for unmapped codes


    # Add mapped columns
    silver_df = silver_df.withColumn("Country_Name", country_name_expr)

    # Drop unnecessary columns
    silver_df = silver_df.drop("Country_Code")

    return silver_df

# Gold Layer
@dlt.table(name="gold_table", comment="Final aggregated and cleaned water quality data")
def gold_table():
    silver_df = dlt.read("silver_table")

    # Removing duplicates and adding new columns
    gold_df = silver_df.dropDuplicates() \
        .withColumn("Start_Date", col("Sampling_Period").substr(1, 10)) \
        .withColumn("End_Date", col("Sampling_Period").substr(-10, 10))
    
    return gold_df

